## System Architecture 12 - Deep Learning Software Programming

## (Part I) Model design and training

Author: Qian Zhao and Noriyuki Kushiro, Department of Computer Science and Networks, Kyushu Institute of Technology, 2020.

---

This jupyter notebook shows how to build a simple neural network to recognize handwritting images.

A workable neural network has a proper network structure and correct weight (parameters) values.

The neural network we used is a simple classical convolutional neural network (CNN) for image classification, the LeNet-5. 

We use MNIST dataset to train LeNet-5 to implement accurate handwritting image recognition.

## Prepare: Python packages imported below are required.
If you run the notebook on our prepared server, just push run button and enjoy.

If you wish to run this notebook on your own computer, you have to prepare python enviroment with these packages installed.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
from time import time
from datetime import datetime
import os
import warnings; warnings.simplefilter('ignore')
print("TensorFlow version: ", tf.__version__)

Choose CPU or GPU to run this notebook (default is CPU here).

CPU: use "os.environ['CUDA_VISIBLE_DEVICES'] = '-1'" line.
GPU: comment out "os.environ['CUDA_VISIBLE_DEVICES'] = '-1'".

Click "Kernel" -> "Restart & Clean output" after change.

In [ ]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #comment out this line if you want to use GPU
if tf.test.gpu_device_name():
    print('GPU found')
    !nvidia-smi
else:
    print("No GPU found, use CPU")

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Step 1. Download [MNIST](http://yann.lecun.com/exdb/mnist/) dataset and prepare data for training 

Prepare MNIST dataset using Keras.datasets automatically.

In [ ]:
%%time

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()


Change and print out tensor shape.

The dataset tensor shape has four dimentions, like (60000, 28, 28, 1)

1st dimension 60000 is the index of data item.

(28, 28, 1) means 28 * 28 pixels image, with color channel of 1 (gray). An RGB color image has 3 color channels.

In [ ]:
%%time

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[:, :, :, np.newaxis]
x_test = x_test[:, :, :, np.newaxis]

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

Preview some image samples of MNIST dataset.

In [ ]:
%matplotlib inline

plt.figure(figsize=(12,10))
x, y = 10, 4
for i in range(40):  
    plt.subplot(y, x, i+1)
    plt.imshow(x_train[i].reshape((28,28)),interpolation='nearest')
plt.show()

By here, we have imported MNIST handwritting data for training and testing our neural network.
There are 60,000 samples for training, which are saved in x_train (input image) and y_train (expecetd label).
There are 10,000 samples for test, which are saved in x_test (input image) and y_test (expecetd label).

## Step 2. Building a classic LeNet-5 nerual network model

Descripting the structure of the classical simple LeNet-5 using Keras frontend modules. Keras provides common neural network layers like Dense, Flatten, Conv2D, AveragePooling2D. These layers can be convinentlly instanced, customized and connected to form various neural networks.

In [ ]:
# LeNet-5 model as example
class LeNet(Sequential):
    def __init__(self, input_shape, nb_classes):
        super().__init__()

        self.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
        self.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
        self.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
        self.add(Flatten())
        self.add(Dense(120, activation='relu'))
        self.add(Dense(84, activation='relu'))
        self.add(Dense(nb_classes, activation='softmax'))

        self.compile(optimizer='adam',
                    loss=categorical_crossentropy,
                    metrics=['accuracy'])

Make a model instance from LeNet class. Then show the structure summary of model.

In [ ]:
%%time

input_shape = x_train[0].shape  # set input tensor shape with the shape of the first x_train data.
num_class = 10                             # set the number of output class. An input image will be classified to 10 numbers (0 ~ 9).
model = LeNet(x_train[0].shape, num_class)
model.summary()

By here, we have finished building the structure of the LeNet-5 neural network.

## Step 3. Train model with dataset

Finally, we use the MNIST dataset to train LeNet-5 model. As shown in the model summary, the LeNet-5 has 44,426 trainable parameters. The target of the training process is to set these parameters to proposer values.

In [ ]:
#setup logging callback function, so we can log the training process
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs, histogram_freq = 1, profile_batch = '5, 10')

Perform model training.

epochs = 5 : train the model for 5 times using the same dataset.

batch_size = 20 : number of data for one training process.

In [ ]:
%%time

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=20, callbacks=[tboard_callback])

Show training result with tensorboard. We can analysis accuracy and performance of neural network model with this tool.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

Save the trained weights to a file, so it can be used in future without retraining.

In [ ]:
model.save_weights('my_lenet_mnist_model.weight')